<a href="https://colab.research.google.com/github/ynklab/cl/blob/main/semantics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

意味表示の導出、モデル検査、定理証明

NLTK: https://www.nltk.org/ は、Pythonプログラミング言語で書かれた英語の自然言語処理のためのライブラリであり、NLTKを使うことで意味表示の導出、モデル検査や定理証明を簡単に記述できる。このコードではNLTKを用いた意味表示の導出、モデル検査、定理証明について簡潔に紹介する。ぜひNLTKの公式ドキュメントhttps://www.nltk.org/howto/semantics.html も参考にしてほしい。

まずは必要なモジュールをimportしよう。


In [1]:
import nltk
from nltk import *
from nltk.sem import logic
from nltk.sem import Valuation, Model
from nltk.sem.logic import LogicParser
from nltk.sem.util import parse_sents

NLTKではfeature based grammarを指定して構文解析・意味解析を行うことができる。ここではhttps://github.com/nltk/nltk_teach/blob/master/examples/grammars/sample_grammars/sem2.fcfg　をfeature based grammarに指定したときの基本的な文の意味合成の実装例を紹介する。Parse:以下が意味合成（SEM=以下が各構成素の意味表示）、Semantics:以下が文の意味表示を示す。


In [36]:
# feature based grammarの読み込み
nltk.download('sample_grammars')
gramfile = 'grammars/sample_grammars/sem2.fcfg'

# 構文解析・意味解析
inputs = ['John sees Mary', 'a dog walks']
parses = parse_sents(inputs, gramfile)
for sent, trees in zip(inputs, parses):
    print("Sentence: %s" % sent)
    for tree in trees:
        print("Parse:\n %s" %tree)
        print("Semantics: %s" %  root_semrep(tree))

Sentence: John sees Mary
Parse:
 (S[SEM=<see(john,mary)>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(john)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(john)>] John))
  (VP[NUM='sg', SEM=<\y.see(y,mary)>]
    (TV[NUM='sg', SEM=<\X y.X(\x.see(y,x))>, TNS='pres'] sees)
    (NP[-LOC, NUM='sg', SEM=<\P.P(mary)>]
      (PropN[-LOC, NUM='sg', SEM=<\P.P(mary)>] Mary))))
Semantics: see(john,mary)
Sentence: a dog walks
Parse:
 (S[SEM=<exists x.(dog(x) & walk(x))>]
  (NP[NUM='sg', SEM=<\Q.exists x.(dog(x) & Q(x))>]
    (Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>] a)
    (Nom[NUM='sg', SEM=<\x.dog(x)>]
      (N[NUM='sg', SEM=<\x.dog(x)>] dog)))
  (VP[NUM='sg', SEM=<\x.walk(x)>]
    (IV[NUM='sg', SEM=<\x.walk(x)>, TNS='pres'] walks)))
Semantics: exists x.(dog(x) & walk(x))


[nltk_data] Downloading package sample_grammars to /root/nltk_data...
[nltk_data]   Package sample_grammars is already up-to-date!


次に、7.2節の例に出てきた2つのモデルS_1, S_2を読み込み、Hanako is young and smartの意味表示smart(hanako)&young(hanako)の真理値をモデル検査で確かめよう。

In [37]:
# モデルの読み込み
S_1 = """
hanako => a
yamada => b
taro => c
smart => {a, b}
young => {a, c}
"""

S_2 = """
hanako => a
yamada => b
taro => c
smart => {a, c}
young => {b, c}
"""

In [38]:
# モデルS_1におけるモデル検査
val = Valuation.fromstring(S_1)
dom = val.domain
m = nltk.Model(dom, val)
g = nltk.sem.Assignment(dom)
m.evaluate('smart(hanako)&young(hanako)', g)

True

In [39]:
# モデルS_2におけるモデル検査
val = Valuation.fromstring(S_2)
dom = val.domain
m = nltk.Model(dom, val)
g = nltk.sem.Assignment(dom)
m.evaluate('smart(hanako)&young(hanako)', g)

False

次に、7.1節で紹介した文間の含意関係、矛盾関係の証明をNLTKの導出原理(resolution)に基づく定理証明で示してみよう。

In [40]:
# Hanako is young and smartとHanako is smartの含意関係の証明
read_expr = nltk.sem.Expression.fromstring
lf1 = read_expr('smart(hanako)&young(hanako)', type_check=True)
lf2 = read_expr('smart(hanako)', type_check=True)
# lf1からlf2を導く証明を導出する
ResolutionProver().prove(lf2, [lf1], verbose=True)

[1] {-smart(hanako)}  A 
[2] {smart(hanako)}   A 
[3] {young(hanako)}   A 
[4] {}                (1, 2) 



True

In [41]:
# There is no womanとA woman is dancingの矛盾関係の証明
lf5 = read_expr('-exists x.(woman(x))', type_check=True)
lf6 = read_expr('exists x.(woman(x)&dance(x))', type_check=True)
# 矛盾関係の場合はlf5とlf6からFalseを導く証明を導出する
ResolutionProver().prove(False, [lf5,lf6], verbose=True)

[1] {-woman(z35)}  A 
[2] {woman(z37)}   A 
[3] {dance(z38)}   A 
[4] {}             (1, 2) 



True

In [43]:
# Some dogs ranとSome animals ranの含意関係の証明
lf7 = read_expr('exists x.(dog(x)&run(x))', type_check=True)
axiom = read_expr('forall x.(dog(x) -> animal(x))', type_check=True) #dogに属する個体はすべてanimalにも属するという公理の補完が必要
lf8 = read_expr('exists x.(animal(x)&run(x))', type_check=True)
ResolutionProver().prove(lf8, [lf7,ax], verbose=True)

[ 1] {-animal(z47), -run(z47)}  A 
[ 2] {dog(z49)}                 A 
[ 3] {run(z50)}                 A 
[ 4] {-dog(z52), animal(z52)}   A 
[ 5] {-animal(z50)}             (1, 3) 
[ 6] {-dog(z52), -run(z52)}     (1, 4) 
[ 7] {animal(z52)}              (2, 4) 
[ 8] {-run(z52)}                (1, 7) 
[ 9] {-run(z52)}                (2, 6) 
[10] {-dog(z52)}                (3, 6) 
[11] {}                         (2, 10) 



True

Hanako is smartの意味表示（論理式）であるsmart(hanako)の項と述語の型を確認してみよう。

In [44]:
# hanakoの型
print(lf2.argument.type)

# smartの型
print(lf2.function.type)

e
<e,?>


いま、tallの型は何も指定していないので、他の型になる可能性を考慮して、型チェッカーでは<e, ?>になっている。tallの型を明示的に語彙項目で定義して、もう一度型を確認しよう。

In [45]:
sig = {'smart': '<e, t>'}
lf2.typecheck(sig)
# smartの型
print(lf2.function.type)

<e,t>


語彙項目を指定して論理式を読みこみ（type_check=Trueで型チェッカーがONになっている）型を確認し、モデル検査や定理証明を行ってみよう。

In [46]:
# 語彙項目の指定
sig = {'john': 'e', 'mary': 'e', 'tom': 'e', 'smart': '<e,t>', 'run': '<e,t>', 'like': '<e,<e,t>>'}

# 論理式の例
examples = [r'run(john)', # John runs
             r'\x.run(x)', # run
             r'john', # John
             r'\x.smart(x)', # smart
             r'\x.like(john,x)(mary)', # 関数適用前John likes Mary
             r'like(john, mary)'] # 関数適用後John likes Mary

# 論理式の読み込み
examples = [read_expr(e, type_check=True) for e in examples]

# 型の確認
for example in examples:
    example.typecheck(sig)
    print(example.type)

t
<e,t>
e
<e,t>
t
t


In [47]:
# モデルの読み込み
S_1 = """
john => a
mary => b
tom => c
smart => {a}
run => {a, b}
like => {(a,b),(c,b)}
"""
val = Valuation.fromstring(S_1)
dom = val.domain
m = nltk.Model(dom, val)
g = nltk.sem.Assignment(dom)


f1 = 'smart(john)' # John is smart
f2 = 'smart(mary)' # Mary is smart
lf1 = read_expr(f1, type_check=True)
lf2 = read_expr(f2, type_check=True)

# モデル検査
print(m.evaluate(f1, g)) # John is smartはS_1のもとで真
print(m.evaluate(f2, g)) # Mary is smartはS_1のもとで偽

# John is smartとMary is smart間の非含意の関係を定理証明
ResolutionProver().prove(lf2, [lf1], verbose=True)

True
False
[1] {-smart(mary)}  A 
[2] {smart(john)}   A 



False

ちなみに、.simplify()はラムダ計算のβ簡約を行う関数である。

In [48]:
x1 = read_expr(r'\x.\y.like(x,y)(john)(mary)').simplify()
x2 = read_expr(r'like(john,mary)')
print(x1 == x2)

x1 = read_expr(r'\x.\y.like(x,y)(mary)(john)').simplify()
x2 = read_expr(r'like(john,mary)')
print(x1 == x2)

True
False
